In [1]:
import pandas as pd
import requests
from tqdm import tqdm

df = pd.read_csv("../data/subset.csv")

df["protein_id"] = ""
df["domain_start"] = None
df["domain_end"] = None
df["protein_sequence"] = ""

def get_uniprot_mapping(pdb_id, chain_id):

    #Get UniProt ID and coverage region for a given PDB ID and chain from PDBe API

    url = f"https://www.ebi.ac.uk/pdbe/api/mappings/uniprot/{pdb_id.lower()}"
    response = requests.get(url)
    if response.status_code != 200:
        print(f" Could not fetch mapping for {pdb_id}")
        return None

    data = response.json()
    mappings = data.get(pdb_id.lower(), {}).get("UniProt", {})

    for uniprot_id, details in mappings.items():
        for region in details.get("mappings", []):
            if region["chain_id"] == chain_id:
                return uniprot_id, int(region["start"]["residue_number"]), int(region["end"]["residue_number"])

    print(f" No mapping found for chain {chain_id} in {pdb_id}")
    return None


def get_uniprot_sequence(uniprot_id):
    #Get the full protein sequence from UniProt

    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta"
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print(f" Could not fetch UniProt sequence for {uniprot_id}")
            return ""
        lines = response.text.splitlines()
        return "".join(lines[1:])
    except Exception as e:
        print(f" Error fetching sequence for {uniprot_id}: {e}")
        return ""


for idx, row in tqdm(df.iterrows(), total=len(df)):
    domain_id = row["domain_id"]
    if len(domain_id) < 5:
        continue  # skip malformed IDs

    pdb_id = domain_id[:4]
    chain_id = domain_id[4]

    mapping = get_uniprot_mapping(pdb_id, chain_id)
    if mapping:
        uniprot_id, start, end = mapping
        sequence = get_uniprot_sequence(uniprot_id)

        df.at[idx, "protein_id"] = uniprot_id
        df.at[idx, "domain_start"] = start
        df.at[idx, "domain_end"] = end
        df.at[idx, "protein_sequence"] = sequence

df.to_csv("../data/subset_with_protein_info.csv", index=False)

  4%|▍         | 42/1000 [00:17<05:28,  2.91it/s]

 Could not fetch mapping for 4g54


  9%|▉         | 94/1000 [00:37<05:23,  2.80it/s]

 No mapping found for chain B in 4a5u


 13%|█▎        | 133/1000 [00:52<04:37,  3.13it/s]

...

 98%|█████████▊| 976/1000 [06:30<00:07,  3.15it/s]

 Could not fetch mapping for 1vw4


 98%|█████████▊| 980/1000 [06:31<00:04,  4.16it/s]

 Could not fetch mapping for 4a17
 Could not fetch mapping for 1vw4


100%|██████████| 1000/1000 [06:39<00:00,  2.50it/s]
